In [1]:
import pandas as pd
import torch
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T
import random
import sys
sys.path.append('../')
import src.util.tigergraph_util as tgu
from src.models.models import NetStatKNN, GNNHandler

## Preprocessing

In [3]:
user_subreddits = pd.read_csv('../data/out/user_subreddit.csv', header=None)
user_subreddits.columns = ['user', 'subreddit', 'times']
pop_subs = set(user_subreddits[['subreddit', 'times']].groupby('subreddit')['times'].count().sort_values(ascending=False).head(25).index)
user_subreddits['subreddit times'] = list(zip(user_subreddits['times'], user_subreddits['subreddit']))
user_subreddits = user_subreddits.groupby('user')['subreddit times'].apply(list)
user_subreddit_map = user_subreddits.apply(lambda x: [s for c, s in sorted(x, reverse=True)]).to_dict()

c:\Users\Scott\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
users = list(user_subreddit_map.keys())
rand_users = random.sample(users, 100)

## Defining Models

### Network Statistics

In [4]:
conn = tgu.connection('../config/tigergraph.json')
df = conn.getVertexDataFrame('user', select='fastrp_embedding')
df = pd.concat([df['v_id'].to_frame(), df['fastrp_embedding'].apply(pd.Series)], axis=1)
df.columns = ['v_id', 'pagerank', 'louvain', 'label_prop', 'degree']
embeddings = pd.read_csv('../data/out/user.csv', header=None)
embeddings = embeddings.rename(columns={0:'v_id'})
embeddings['v_id'] = embeddings['v_id'].astype(str)
df['v_id'] = df['v_id'].astype(str)
user_data = df.merge(embeddings, on='v_id', how='inner')
subreddit_data = pd.read_csv('../data/out/subreddit.csv', header=None)
user_subreddit = pd.read_csv('../data/out/user_subreddit.csv', header=None)
reddit_graph = NetStatKNN(conn)
reddit_graph.fit(user_data, subreddit_data, user_subreddit, 10)

## Example Recommendation-making

NameError: name 'rand_users' is not defined